In [69]:
import urllib.request as req
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

import pymysql

In [5]:
url="http://www.yes24.com/Product/Search?domain=ALL&query=python"
res=req.urlopen(url)
bs=BeautifulSoup(res,'html.parser')

In [76]:
book_list = bs.select('#yesSchList >li')

result=[]

for idx, book in enumerate(book_list):
    image_url = book.find(class_='img_bdr').img['data-original']
    title= book.find(class_='gd_name').string
    info = book.find(class_='gd_nameE').string if book.find(class_='gd_nameE') else ''
    author=book.find(class_='info_auth').a.string
    publisher=book.find(class_='info_pub').a.string
    price=book.find(class_='txt_num').em.string
    date = book.find(class_='info_date').string
    
    result.append((title, info, author, publisher, price, date))
    # image_save(image_url,idx)

In [64]:
df = pd.DataFrame(result)
df.columns=['image', 'title', 'info', 'author', 'pub', 'price','date']
df.to_csv('books.csv', index=False)

In [49]:
def image_save(url,i):
    save_name='data/book_images/image'+str(i)+'.jpg'
    req.urlretrieve(url,save_name)
    print(save_name,'이(가) 저장되었습니다')

In [77]:
conn = pymysql.connect(host='localhost',
               user='root', password='1234', 
               db='pydb',charset='utf8')
cursor=conn.cursor()
sql="insert into book (title, info, author, pub, price, date) values(%s,%s,%s,%s,%s,%s)"
cursor.executemany(sql,result)
conn.commit()
conn.close()